In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
file_path = "datagame-2023/"
train = pd.read_parquet(f"{file_path}/label_train_source.parquet")
test = pd.read_parquet(f"{file_path}/label_test_source.parquet")
target = pd.read_parquet(f"{file_path}/label_train_target.parquet")
meta_song = pd.read_parquet(f"{file_path}/meta_song.parquet")
trainR = train["song_id"]
targetR = target["song_id"]
testR = test["song_id"]
rank = pd.concat([trainR,targetR,testR])

In [ ]:
r = rank.value_counts()
rank_result = r.rank(ascending=False, method='dense')
ranked = pd.DataFrame({'Value': r.index, 'Count': r.values, 'Rank': rank_result.values})


In [ ]:
result = {}
def model():
    # baseline ....
    for i in range(0,len(test),20):
        tmp = []
        rng = 0
        if i%1000==0 and i>=1000:print(i)
        for j in range(20):
            idx = i+j
            rng = max(rng,int(ranked[ranked["Value"]==test["song_id"][idx]]["Rank"].iloc[0]))
            song_id = test["song_id"][idx]
            if song_id in tmp:
                # print(len(tmp))
                try:
                    temp = result[test["session_id"][idx]]
                    if song_id not in temp:
                        # print("jizz")
                        result[test["session_id"][idx]].append(song_id)
                except:
                    result[test["session_id"][idx]] = [song_id]
                    # result[test["session_id"][idx]].append(test["song_id"][idx])
            tmp.append(song_id)
        if rng==0: rng = len(ranked) 
        try:
            length = len(result[test["session_id"][idx]])
            if length<5:
                # print("wtf")
                # print(len(result[test["session_id"][idx]]))
                n = 5-length
                rnd = ranked[:rng]
                # print(n==5,i)
                rnd = rnd[~rnd["Value"].isin(result[test["session_id"][idx]])]
                # print("wwwwwwwwwtttttf")
                result[test["session_id"][idx]].extend(rnd.sample(n)["Value"].to_list())
                # print(len(result[test["session_id"][idx]]),i)
            else:
                pass
                # print(len(result[test["session_id"][idx]]),i)
        except:
            if rng>5:
                rnd = ranked[:rng]
            else:
                rnd = ranked[:]
                # print(n==5,i)
            n=5
            # print(len(rnd))
            result[test["session_id"][idx]]= rnd.sample(n)["Value"].to_list()
            # print(len(result[test["session_id"][idx]]),i)